# 实验一
自定义VGG16网络，并在Kaggle猫/狗数据集上进行训练和测试

## 1.加载keras模块

In [38]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
import numpy as np

### 定义CNN网络结构



In [44]:
img_width, img_height = 150, 150
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
 #定义VGG16结构 
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation="relu", input_shape=input_shape))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32,kernel_size=(3,3),activation="relu", input_shape=input_shape))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,kernel_size=(3,3),activation="relu", input_shape=input_shape))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(64,activation="sigmoid"))
model.add(Dropout(0.1))
model.add(Dense(1, activation="sigmoid"))

#在compile函数里定义loss函数、优化器和metrics
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### 查看model架构



In [45]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 18496)            

### 定义ImageDataGenerator


In [46]:
train_data_dir = r'/home/zhou/zhou/实验10/dogs-vs-cats/train'

validation_data_dir = r'/home/zhou/zhou/实验10/dogs-vs-cats/validation'
nb_train_samples = 10835
nb_validation_samples = 4000
epochs = 1
batch_size = 20

#定义ImageDataGenerator为生成training和validation samples
train_datagen = ImageDataGenerator(zoom_range=0.2,rescale=1./255,shear_range=0.2,horizontal_flip=True)
test_datagen =  ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [47]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/1
541/541 [==============================] - 243s 449ms/step - loss: 0.6412 - accuracy: 0.6256 - val_loss: 0.3770 - val_accuracy: 0.7235


### 使用训练后模型预测图像





In [48]:
import cv2
img = cv2.resize(cv2.imread(r'C:\Users\coffe\Desktop\dogs-vs-cats\test\7.jpg'), (img_width, img_height)).astype(np.float32)

x = img_to_array(img)

x = np.expand_dims(x, axis=0)

#x = preprocess_input(x)

score = model.predict(x)


print(score)

ModuleNotFoundError: No module named 'cv2'